## MODELO FINAL

·····································································································································································································································································
Para nuestro modelo final vamos a utilizar un modelo de SVM radial que utiliza busqueda aleatoria para la optimización de hiperparámetros, utilizando en vez de valores particulares, valires con distribuciones estadísticas(loguniform).
·····································································································································································································································································

·····································································································································································································································································
Al utiliar otro notebook hemos tenido que volver a realizar algunos procedimientos, como la eliminación de atributos y el modelo SVM_RADIAL con distribuciones estadísticas(el modelo seleccionado).
·····································································································································································································································································

----------------------------------------------------------------------------------------------------------------
## IMPORTS
----------------------------------------------------------------------------------------------------------------

In [12]:
import pandas as pd
import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_decision_regions
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from scipy.stats import uniform
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
from scipy.stats import randint as sp_randint
from sklearn.inspection import permutation_importance
import multiprocessing
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

----------------------------------------------------------------------------------------------------------------
## CARGA Y ELIMINACIÓN DATOS
----------------------------------------------------------------------------------------------------------------

In [13]:
# Cargamos los datos en un data frame 
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Eliminar todos los datos que no corresponden a la localización 13

wind_ava_13 = [col for col in wind_ava.columns if col == 'datetime' or col == 'energy' or col.endswith('.13')]
df_filtered = wind_ava[wind_ava_13]

df_filtered.head()

,datetime,energy,p54.162.13,p55.162.13,cape.13,p59.162.13,lai_lv.13,lai_hv.13,u10n.13,v10n.13,...,t2m.13,stl2.13,stl3.13,iews.13,inss.13,stl4.13,fsr.13,flsr.13,u100.13,v100.13
0,2005-01-02 18:00:00,402.71,2.510824e+06,9.186295,13.527577,1.386937e+06,2.344111,2.432983,-0.757587,-1.922799,...,280.473098,281.042026,281.462478,-0.057958,-0.138650,284.684755,0.404731,-5.927092,-1.780562,-4.443617
1,2005-01-03 00:00:00,696.80,2.513173e+06,8.849569,6.896412,1.153526e+06,2.343719,2.432838,-1.412620,-1.403011,...,278.286616,280.747406,281.486541,-0.103576,-0.083050,284.667948,0.404920,-5.913881,-3.743344,-3.129469
2,2005-01-03 06:00:00,1591.15,2.509627e+06,7.924080,4.774439,1.098754e+06,2.343300,2.432704,-2.290185,-0.754580,...,277.206490,280.114863,281.487095,-0.165721,-0.036241,284.651914,0.405704,-5.908272,-5.097203,-1.157748
3,2005-01-03 12:00:00,1338.62,2.510571e+06,6.922709,0.000000,1.076021e+06,2.342830,2.432514,-3.497855,1.271028,...,280.926600,279.991138,281.472435,-0.275550,0.098192,284.636266,0.403967,-5.961995,-4.500835,1.502478
4,2005-01-03 18:00:00,562.50,2.505664e+06,6.646282,0.000000,1.070830e+06,2.342437,2.432369,-0.971249,0.553060,...,277.363875,280.576898,281.473265,-0.056553,0.041844,284.620232,0.403808,-5.987860,-3.392324,2.131114


In [14]:
df_filtered = df_filtered.drop('datetime', axis=1)

In [15]:
numeric_columns = df_filtered.select_dtypes(include=['float64', 'int64'])
varianza = numeric_columns.var()
porcentaje_umbral = 0.0000000000001  # Eliminar variables cuya varianza sea menos del 0.0000000000001% 
umbral_varianza = varianza.max() * porcentaje_umbral
variables_a_eliminar = varianza[varianza< umbral_varianza].index.tolist()
print("Variables a eliminar:", variables_a_eliminar)
variance = df_filtered.select_dtypes(include=['float64', 'int64']).var()
low_variance_columns = variance[variance < 0.2].index
df_filtered = df_filtered.drop(columns=low_variance_columns)
print(df_filtered.shape)

Variables a eliminar: ['lai_lv.13', 'lai_hv.13', 'iews.13', 'inss.13', 'fsr.13', 'flsr.13']
(4748, 17)


In [16]:
y_df = df_filtered['energy']
print(y_df)
X_df = df_filtered.drop('energy', axis=1)
print(X_df)

0        402.71
1        696.80
2       1591.15
3       1338.62
4        562.50
         ...   
4743    1280.13
4744     855.00
4745     117.06
4746     516.96
4747     867.07
Name: energy, Length: 4748, dtype: float64
        p54.162.13  p55.162.13    cape.13    p59.162.13   u10n.13   v10n.13  \
0     2.510824e+06    9.186295  13.527577  1.386937e+06 -0.757587 -1.922799   
1     2.513173e+06    8.849569   6.896412  1.153526e+06 -1.412620 -1.403011   
2     2.509627e+06    7.924080   4.774439  1.098754e+06 -2.290185 -0.754580   
3     2.510571e+06    6.922709   0.000000  1.076021e+06 -3.497855  1.271028   
4     2.505664e+06    6.646282   0.000000  1.070830e+06 -0.971249  0.553060   
...            ...         ...        ...           ...       ...       ...   
4743  2.385772e+06   14.413342  24.971074  3.905586e+06  4.595786  5.563189   
4744  2.385708e+06   12.802883  60.248874  2.745694e+06  4.894838  5.577096   
4745  2.379049e+06   10.915997   3.145067  2.254171e+06  0.958361  0.9

----------------------------------------------------------------------------------------------------------------
## MODELO FINAL
----------------------------------------------------------------------------------------------------------------

In [17]:
def rmse(y_test, y_test_pred):
  """ Este es mi cálculo del error cuadrático medio """
  return np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

··········································································································································································································································································
ENTRENAMIENTO MODELO
··········································································································································································································································································

In [18]:
#Seleccionar la mejor alternativa de las evaluadas en los puntos anteriores. Modelo seleccionado.
outer = KFold(n_splits=5, shuffle=True, random_state=23)
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df,test_size=0.33, random_state= 23)
budget = 50
inner = KFold(n_splits=3, shuffle=True, random_state=23)
svr_radial = SVR()
escalar = StandardScaler()
pipe_regr_radial_def = Pipeline([
    ('escalar', escalar),
    ('SVM', svr_radial)])
# Search space
param_grid = {'SVM__C': loguniform(1e-1, 1e4),
              'SVM__gamma': loguniform(1e-5, 1e1)}

hpo_regr_radial = RandomizedSearchCV(pipe_regr_radial_def,
                            param_grid,
                            scoring='neg_mean_squared_error',
                            cv=inner,
                            n_jobs=4, verbose=0,
                            n_iter=budget
                        )
start_time = time.time()
np.random.seed(23)
hpo_regr_radial.fit(X=X_train, y=y_train)
rmse_svm_dist = rmse(y_test, hpo_regr_radial.predict(X=X_test))
training_time_svm_dist_random = time.time() - start_time
print("Tiempo de entrenamiento:", training_time_svm_dist_random)


Tiempo de entrenamiento: 18.068214416503906


··········································································································································································································································································
Modelo final y evaluar el desempeño  futuro  del  modelo  (evaluación  outer).
··········································································································································································································································································

In [19]:
modelo_final = hpo_regr_radial.best_estimator_

In [20]:
# Estimar  el  rendimiento  /  desempeño  futuro  del  modelo  (evaluación  outer).
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

scores = cross_val_score(modelo_final,
                          X_train,
                          y_train,
                          scoring=make_scorer(rmse),
                          cv=outer)

# Print the scores or perform other evaluations
print("Mean squared error:", scores.mean())


Mean squared error: 369.24928844874387


··········································································································································································································································································
Entrenar modelo final y guardarlo en un fichero(modelo_final.pkl)
··········································································································································································································································································

In [21]:
import pickle
#Entrenar el modelo final. Guardarlo en un fichero (llamado «modelo_final.pkl»).
modelo_final.fit(X_df, y_df)
with open("modelo_final.pkl", "wb") as f:
    pickle.dump(modelo_final, f)


··········································································································································································································································································
Utilizar el modelo final para obtener predicciones para el conjunto de datos de la competición.  Se guardan el predicciones.csv
··········································································································································································································································································

In [22]:
#Utilizar el modelo final para obtener predicciones para el conjunto de datos de la 
#competición.  Guardar estas predicciones en un fichero (llamado «predicciones.csv»)
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
# Cargar el conjunto de datos de competición
wind_comp = pd.read_csv('wind_comp.csv.gz', compression="gzip")

# Filtrar los datos para retener solo la localización 13
wind_comp_13 = [col for col in wind_comp.columns if col.endswith('.13')]
df_comp_filtered = wind_comp[wind_comp_13]


# Calcular la varianza de las características numéricas
numeric_columns = df_comp_filtered.select_dtypes(include=['float64', 'int64'])
variance = numeric_columns.var()

# Definir umbral de varianza para eliminar características con varianza muy baja
porcentaje_umbral = 0.0000000000001
umbral_varianza = varianza.max() * porcentaje_umbral

# Eliminar características con varianza muy baja
variables_a_eliminar = variance[variance < umbral_varianza].index.tolist()
df_comp_filtered = df_comp_filtered.drop(columns=variables_a_eliminar)

# Cargar el modelo final desde el archivo modelo_final.pkl
with open("modelo_final.pkl", "rb") as f:
    model_final = joblib.load(f)

# Hacer predicciones en el conjunto de datos de competición
predictions = model_final.predict(df_comp_filtered)

# Guardar las predicciones en un archivo CSV llamado "predicciones.csv"
predictions_df = pd.DataFrame(predictions, columns=["energy"])
predictions_df.to_csv("predicciones.csv", index=False)